<a href="https://colab.research.google.com/github/FranciscoReveriano/Facial-Expression-Recognition/blob/master/train_TwoConvolutionalLayer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Original Model

##Train The Dataset

### Import Necessary Libaries

In [0]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import time 

import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D
from keras.layers.core import Dense, Dropout, Flatten, Activation

###Check for GPU

In [50]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

config = tf.ConfigProto()
sess = tf.Session(config=config)
keras.backend.set_session(sess)

Found GPU at: /device:GPU:0


### Select Settings

In [0]:
num_classes = 7
batch_size = 256
epochs = 50
VERBOSE = 1
VALIDATION_SPLIT = 0.2

### Process Images

In [52]:
start = time.time() 
with tf.device('/GPU:0'):
  with open("fer2013.csv") as f:
      content = f.readlines()
  lines = np.array(content)
  num_of_instances = lines.size
end = time.time()
print("Number of Images: ", num_of_instances)
print("Pixel number of each image: ", len(lines[1].split(",")[1].split(" ")))
print("Time to prepare images = ", end-start)

Number of Images:  35888
Pixel number of each image:  2304
Time to prepare images =  1.8909986019134521


### Train the Model

In [53]:
start = time.time()
with tf.device('/GPU:0'):
  x_train, y_train = [], []

  for i in range(1, num_of_instances):
      emotion, img, usage = lines[i].split(",")
      val = img.split(" ")
      pixels = np.array(val, 'float32')
      emotion = keras.utils.to_categorical(emotion, num_classes)
      if 'Training' in usage:
          y_train.append(emotion)
          x_train.append(pixels)

  x_train = np.array(x_train, 'float32')
  y_train = np.array(y_train, 'float32')
  x_train /= 255
  x_train = x_train.reshape(x_train.shape[0], 48, 48, 1)
  x_train = x_train.astype('float32')
  print(x_train.shape[0], 'train samples')

  model = Sequential()


  # Two Convolutional Layer
  model.add(Conv2D(32, (3, 3),  activation='relu', padding='same', input_shape=(48, 48, 1)))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Dropout(0.2))
  model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Dropout(0.2))
  model.add(Flatten())
  model.add(Dense(512, activation='relu'))
  model.add(Dropout(0.5))

  model.add(Dense(num_classes, activation='softmax'))
  model.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.Adam(), metrics=['accuracy'])
  # model.summary()
  history = model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, verbose=VERBOSE, validation_split=VALIDATION_SPLIT)

  model_json = model.to_json()
  with open("model.json", "w") as json_file:
      json_file.write(model_json)
  model.save_weights("model.h5")
  print("Saved model to disk")


end = time.time()
print("Saved model to disk")
print("Total Runtime To Train Model = ", end-start)

28709 train samples
Train on 22967 samples, validate on 5742 samples
Epoch 1/50
22967/22967 [==============================] - 4s 164us/step - loss: 1.8155 - acc: 0.2532 - val_loss: 1.7350 - val_acc: 0.3208
Epoch 2/50
22967/22967 [==============================] - 3s 122us/step - loss: 1.6753 - acc: 0.3374 - val_loss: 1.6095 - val_acc: 0.3798
Epoch 3/50
22967/22967 [==============================] - 3s 121us/step - loss: 1.5812 - acc: 0.3896 - val_loss: 1.5036 - val_acc: 0.4284
Epoch 4/50
22967/22967 [==============================] - 3s 121us/step - loss: 1.5033 - acc: 0.4213 - val_loss: 1.4362 - val_acc: 0.4542
Epoch 5/50
22967/22967 [==============================] - 3s 122us/step - loss: 1.4396 - acc: 0.4452 - val_loss: 1.3843 - val_acc: 0.4828
Epoch 6/50
22967/22967 [==============================] - 3s 122us/step - loss: 1.3916 - acc: 0.4663 - val_loss: 1.3635 - val_acc: 0.4869
Epoch 7/50
22967/22967 [==============================] - 3s 122us/step - loss: 1.3506 - acc: 0.4851 - 

##Test the Dataset

###Import Necessary Libraries

In [0]:
import numpy as np
import matplotlib.pyplot as plt
import keras
import time
from keras.models import model_from_json

In [55]:
start = time.time()
num_classes = 7
with tf.device('/GPU:0'):
  with open("fer2013.csv") as f:
      content = f.readlines()
  lines = np.array(content)
  num_of_instances = lines.size

  x_test, y_test = [], []

  for i in range(1, num_of_instances):
      emotion, img, usage = lines[i].split(",")
      val = img.split(" ")
      pixels = np.array(val, 'float32')
      emotion = keras.utils.to_categorical(emotion, num_classes)
      if 'PublicTest' in usage:
          y_test.append(emotion)
          x_test.append(pixels)

  x_test = np.array(x_test, 'float32')
  y_test = np.array(y_test, 'float32')
  x_test /= 255
  x_test = x_test.reshape(x_test.shape[0], 48, 48, 1)
  x_test = x_test.astype('float32')
  print(x_test.shape[0], 'test samples')

  json_file = open('model.json', 'r')
  loaded_model_json = json_file.read()
  json_file.close()
  loaded_model = model_from_json(loaded_model_json)
  loaded_model.load_weights("model.h5")
  print("Loaded model from disk")
  loaded_model.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.Adam(), metrics=['accuracy'])


def emotion_analysis(emotions):
    objects = ('angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral')
    y_pos = np.arange(len(objects))
    plt.bar(y_pos, emotions, align='center', alpha=0.5)
    plt.xticks(y_pos, objects)
    plt.ylabel('percentage')
    plt.title('emotion')
    plt.show()

evaluation = loaded_model.evaluate(x_test, y_test, batch_size=256, verbose=1)
end = time.time()
print('Summary: Loss over the test data set: %.2f, Accuracy: %.2f' % (evaluation[0], evaluation[1]))
print("Total Time in Testing Model: ", end-start)

3589 test samples
Loaded model from disk
3589/3589 [==============================] - 0s 125us/step
Summary: Loss over the test data set: 1.71, Accuracy: 0.56
Total Time in Testing Model:  15.438267469406128
